# Data preprocessing
This notebook have all the previous data preparation pipeline. It is not needed as intermediate data files are provided.


## Pfam domain dataset 

The starting point is Pfam v32 domains dataset, which are already cropped from the complete proteins. As there are more than 1M sequences, a smaller dataset will be created for quicker tests.

Additional details and full dataset in https://github.com/sinc-lab/llm4pfam. 

In [4]:
import os
import numpy as np
import pandas as pd

# download full dataset
!gdown 1eLfQo-xSSrtFGgbp9ArPblQteiJwOtST #
!tar -xvf data.tar.gz
!rm data.tar.gz

In [4]:
def read_data(path):
    shards = []
    for fn in os.listdir(path):
        with open(os.path.join(path, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    return pd.concat(shards)


data = []

for part in ["train", "dev", "test"]:
    df = read_data(f"data/clustered/{part}/")
    df["part"] = part
    df["sequence_name"] = df.sequence_name.replace("/", "-")
    df["family"] = df["family_accession"].str.split(".").str[0]
    data.append(df.drop(columns=["family_id", "family_accession", "aligned_sequence"]))

data = pd.concat(data)
!rm -r data/clustered/
print("Number of sequences in the dataset:", len(data))
data.head()

Number of sequences in the dataset: 1339083


,sequence_name,sequence,part,family
0,L8GA85_PSED2/604-855,PNKIEHTIAWGRELFESYFVKPAETVNLYLSQPNYINTTLKQGGNE...,train,PF10585
1,R7FMX8_9CLOT/502-554,VIIVIIFGIFLTLVISKILSKTILKGMPSSMILELPPYRKPQFGKI...,train,PF07664
2,R5XBV6_9FIRM/8-110,SALGVIFAVTAVLSFCGKSANLIAGYNTASEEEKAKYDEKKLCNGL...,train,PF12650
3,Q6LXX1_METMP/35-253,LTVGVCAQYPPTIYEENGNLAGFDFDLMNEIAKRMNLNTDFKIYNS...,train,PF00497
4,E3J1Y8_FRAIE/399-483,QVSVAVYNGSMTKGLASKVTTALVGKGFQATTAGNADALTYTTSRV...,train,PF13399


In [8]:
# Get a subset of families from Pfam, reducing also the number of sequences.

families = pd.read_csv("data/pfam_some_families.csv")
data = data[data.family.isin(families.PF)]
data.set_index("sequence_name", inplace=True)
data.to_csv("data/pfam_some_sequences.csv")
data.shape

(80211, 3)

#### Get similarity within each family with blast

Install instructions:

    https://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ or
    sudo apt-get install ncbi-blast+

In [ ]:

!mkdir data/fam_dist
!mkdir tmp
for fam in data.family.unique():
    with open("tmp/seqs.fasta", "w") as f:
        df = data[data.family==fam]
        for k in range(len(df)):
            f.write(f">{df.iloc[k].name}\n{df.iloc[k].sequence}\n")
    !makeblastdb -in tmp/seqs.fasta -dbtype prot -out tmp/db
    !blastp -query tmp/seqs.fasta -num_threads 8 -db tmp/db -out data/fam_dist/{fam}.txt -outfmt 6
    !rm tmp/*

## Computing embeddings


In [1]:
import torch as tr
DEVICE = "cuda"

MAX_LEN = 1022
EMB_NAME = "esm2_t30_150M_UR50D"
LAST_LAYER = 30

#EMB_NAME = "esm2_t33_650M_UR50D"
# LAST_LAYER = 33

# https://github.com/facebookresearch/esm#available-models
model, alphabet = tr.hub.load("facebookresearch/esm:main",
                              EMB_NAME)
model.eval()
model.to(DEVICE)
batch_converter = alphabet.get_batch_converter()
model

Using cache found in /home/lbugnon/.cache/torch/hub/facebookresearch_esm_main


ESM2(
  (embed_tokens): Embedding(33, 640, padding_idx=1)
  (layers): ModuleList(
    (0-29): 30 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=640, out_features=640, bias=True)
        (v_proj): Linear(in_features=640, out_features=640, bias=True)
        (q_proj): Linear(in_features=640, out_features=640, bias=True)
        (out_proj): Linear(in_features=640, out_features=640, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=640, out_features=2560, bias=True)
      (fc2): Linear(in_features=2560, out_features=640, bias=True)
      (final_layer_norm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=600, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((640,), eps=1e-05, elementw

In [2]:
import pandas as pd
import numpy as np

# load sequence datasets
df = pd.read_csv("data/pfam_some_sequences.csv", index_col="sequence_name")

def get_embedding(seq, aggregate=False):
    """Recibe una secuencia, devuelve un tensor de MxL donde M es el tamaño
    del embedding y L la longitud de la secuencia. Si aggregate=True, promedia
    la representación a lo largo de la secuencia devolviendo un vector M"""

    # Recorta el dominio si supera la capacidad del LLM
    center = len(seq)//2
    start = max(0, center - MAX_LEN//2)
    end = min(len(seq), center + MAX_LEN//2)
    seq = seq[start:end]

    # Formato requerido por el tokenizador, se pueden procesar  por lotes
    # en paralelo
    x = [(0, seq)]

    with tr.no_grad():
        _, _, tokens = batch_converter(x)
        emb = model(tokens.to(DEVICE), repr_layers=[LAST_LAYER],
                    return_contacts=True
                    )["representations"][LAST_LAYER].detach().cpu().squeeze()
    
    emb = emb.permute(1,0) # [L, M] -> [M, L], por conveniencia mas adelante
    emb = emb.half() # reduce memory usage

    if aggregate:
        emb = np.array(emb.mean(dim=1))

    return emb

Now we process all the sequences (this can be more efficient with larger GPUs). The process takes about 1h.

In [3]:
from tqdm.notebook import tqdm
import pickle

!mkdir embeddings

embeddings =  {}
for i in tqdm(range(len(df))):
    seq_name = df.iloc[i].name
    # get average embedding for the sequence
    embeddings[seq_name] = get_embedding(df.iloc[i].sequence, aggregate=True) 
    
pickle.dump(embeddings, open(f"embeddings/{EMB_NAME}.pk", "wb"))

mkdir: no se puede crear el directorio «embeddings»: El fichero ya existe


  0%|          | 0/80211 [00:00<?, ?it/s]

AMPLIFY

In [ ]:
# paper: https://www.biorxiv.org/content/10.1101/2024.06.20.599739v2.full.pdf
import os
!git clone https://github.com/chandar-lab/AMPLIFY.git
os.chdir("AMPLIFY")
!pip install --editable .[dev] 
!python3 -m pytest

In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer

#EMB_NAME = "AMPLIFY_120M"
EMB_NAME = "AMPLIFY_350M"

# Load AMPLIFY and tokenizer
model = AutoModel.from_pretrained(f"chandar-lab/{EMB_NAME}", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(f"chandar-lab/{EMB_NAME}", trust_remote_code=True)

# Move the model to GPU (required due to Flash Attention)
model = model.to("cuda")

In [ ]:
from tqdm.notebook import tqdm
import pickle
import torch as tr 
import pandas as pd

# load sequence datasets
df = pd.read_csv("data/pfam_some_sequences.csv", index_col="sequence_name")

def get_embedding(seq, aggregate=False):
    
    with tr.no_grad():
        input = tokenizer.encode(seq, return_tensors="pt")
        input = input.to("cuda")
        output = model(input, output_hidden_states=True)
        output = output.hidden_states[-1].squeeze().cpu() # [1, seq_len, 640]
        if aggregate:
            output = output.mean(dim=0).numpy() # average over all sequence tokens 
 
    return output

embeddings =  {}
for i in tqdm(range(len(df))):
    seq_name = df.iloc[i].name
    # get average embedding for the sequence
    embeddings[seq_name] = get_embedding(df.iloc[i].sequence, aggregate=True) 
    
    
pickle.dump(embeddings, open(f"embeddings/{EMB_NAME}.pk", "wb"))

### PeptideCLM

In [3]:
#!pip install torch transformers datasets SmilesPE pandas
from transformers import AutoModelForSequenceClassification
from peptideclm_tokenizer.tokenizer import SMILES_SPE_Tokenizer
DEVICE = "cuda"

model_name = 'aaronfeller/PeptideCLM-23.4M-all'
tokenizer = SMILES_SPE_Tokenizer('peptideclm_tokenizer/new_vocab.txt', 'peptideclm_tokenizer/new_splits.txt')
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
model.to(DEVICE);

Some weights of RoFormerForSequenceClassification were not initialized from the model checkpoint at aaronfeller/PeptideCLM-23.4M-all and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch as tr  
from tqdm.notebook import tqdm
import pandas as pd
import pickle 
smiles = pd.read_csv("data/smiles_pampa.csv")

embeddings =  {}
for k in tqdm(range(len(smiles))):
    with tr.no_grad():
        input = tokenizer.encode(smiles.iloc[k].SMILES, return_tensors="pt")
        input = input.to(DEVICE)
        output = model(input, output_hidden_states=True).hidden_states[-1].squeeze().cpu()
    # get average embedding for the sequence
    embeddings[k] = output.mean(dim=0).numpy() 
    
    
